# Combining `CurveFits` objects

We can also combine together `CurveFits` objects for distinct serum/viruses/replicates using the `CurveFits.combineCurveFits` method.
This is useful if you are fitting large datasets in chunks, and then want to combine all the results.

Here is an example/test:

In [1]:
import pandas as pd

import neutcurve

Read in the data:

In [2]:
fi6v3_datafile = "Doud_et_al_2018-neutdata.csv"
data = pd.read_csv(fi6v3_datafile)

Create a `CurveFits` object with all the data:

In [3]:
fits = neutcurve.CurveFits(data)
_ = fits.fitParams(average_only=False)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Now split the data to make different `CurveFits` which we can combine:

In [4]:
# split the data in half
data1 = data[(data["replicate"] == 1) | (data["serum"] == "H17-L19")]
data2 = data[(data["replicate"] != 1) & (data["serum"] != "H17-L19")]

# make a second split that is **not** disjoint from the first one
data2_invalid = data[data["replicate"] != 1]

# make fits to each of these
fit1 = neutcurve.CurveFits(data1)
_ = fit1.fitParams(average_only=False)
fit2 = neutcurve.CurveFits(data2)
_ = fit2.fitParams(average_only=False)
fit2_invalid = neutcurve.CurveFits(data2_invalid)
_ = fit2_invalid.fitParams(average_only=False)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)
/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Combine two fits using `CurveFits.combineCurveFits` that should yield an object the same as `fits`:

In [5]:
combined_fits = neutcurve.CurveFits.combineCurveFits([fit1, fit2])

pd.testing.assert_frame_equal(
    fits.fitParams(average_only=False),
    combined_fits.fitParams(average_only=False),
)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Make sure we cannot combine two fits with overlapping entries, this should give an error due to shared serum/virus/replicates:

In [6]:
# NBVAL_RAISES_EXCEPTION

neutcurve.CurveFits.combineCurveFits([fit1, fit2_invalid])

ValueError: duplicated sera/virus/replicate in `curvefits_list`